In [1]:
#Importing Python Libraries
import ast
import glob
import re
from pathlib import Path

import astor
import pandas as pd
import spacy
from tqdm import tqdm
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
# #pip install nmslib
#import nmslib
import numpy
import re
#python -m spacy download en_core_web_sm
EN = spacy.load('en_core_web_sm')

In [ ]:
#Downloading the dataset
# ! wget https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/python.zip
# ! unzip python.zip

In [3]:
#Reading files from databases
path = Path('.')
def jsonl_list_to_dataframe(file_list, columns=None):
    """Load a list of jsonl.gz files into a pandas DataFrame."""
    return pd.concat([pd.read_json(f,
                                   orient='records', 
                                   compression='gzip',
                                   lines=True)[columns] 
                      for f in file_list], sort=False)

def get_dfs(path):
    """Grabs the different data splits and converts them into dataframes"""
    dfs = []
    for split in ["train", "valid", "test"]:
        files = sorted((path/split).glob("**/*.gz"))
        df = jsonl_list_to_dataframe(files, ["code", "docstring","code_tokens","docstring_tokens","url"])
        dfs.append(df)
        
    return dfs


def tokenize_docstring(text):
    "Apply tokenization using spacy to docstrings."
    tokens = EN.tokenizer(text)
    return [token.text.lower() for token in tokens if not token.is_space]


def tokenize_code(text):
    "A very basic procedure for tokenizing code strings."
    return RegexpTokenizer(r'\w+').tokenize(text)

df_trn, df_val, df_tst = get_dfs(path/"python/final/jsonl")

#Subsample
df_trn_sub = df_trn[:10000]
print(df_trn_sub.shape)
docstring = list(df_trn_sub['docstring'].values)
code = list(df_trn_sub['code'].values)
#print(code[0])
codeToken = []
docToken = []

for singleCode,singleDoc in tqdm(zip(code,docstring)):
  codeToken.append(' '.join(tokenize_code(singleCode)))
  docToken.append(' '.join(tokenize_docstring(singleDoc)))
df_trn_sub['code token'] = codeToken
df_trn_sub['doc token'] = docToken
#Clean Code Token
code_token = list(df_trn_sub['code_tokens'].values)
code_token_list = []
for code in code_token:
  code_token_list.append(' '.join(code))

import re
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("","",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

df_idf = pd.DataFrame()
df_idf['text'] = code_token_list
df_idf['text'] = df_idf['text'].apply(lambda x:pre_process(x))
clean_code = list(df_idf['text'].values)
clean_code_token = []
for code in clean_code:
  clean_code_token.append(code.split(' '))
df_trn_sub['clean_code'] = clean_code_token


#Original Tokens
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()
data_corpus = list(df_trn_sub.clean_code.values)
document_corpus = list(df_trn_sub.docstring.values)
buffer = []
for data in data_corpus:buffer.append(' '.join(data))
vocabulary=vectorizer.fit(buffer)
X= vectorizer.transform(buffer)
# print('Code Tokens : ',vocabulary.get_feature_names())
#print('Word Embeddings : ',X.toarray())
baseData = X.toarray()


import numpy as np
from scipy.spatial import distance
from sklearn.metrics import ndcg_score
from tqdm import tqdm
testDocstring = list(df_tst.docstring.values)
filteredDocstring = []
for doc in testDocstring:
    buffer = doc.split(' ')
    if len(buffer) <10 :filteredDocstring.append(doc)
    #filteredDocstring.append(doc)
    
filteredDocstring = filteredDocstring[:400]


from sklearn.metrics.pairwise import cosine_similarity
closest_doc_indexes = {}
testingVector = vectorizer.transform(filteredDocstring)
doctrainingVector = vectorizer.transform(document_corpus)
for index, test_doc_embd in tqdm(enumerate(testingVector)):
  test_doc_embd = np.reshape(test_doc_embd,(1,-1))
  similarities = cosine_similarity(test_doc_embd,doctrainingVector)[0]
  
  most_similar_train_doc = np.argmax(similarities)
  closest_doc_indexes[index] = most_similar_train_doc

#print(closest_doc_indexes)


result = []
ndcga = []
mrr = []
for doc in tqdm(filteredDocstring):
    testingString = [doc]
    testingVector = vectorizer.transform(testingString)
    testingVector = testingVector.toarray()
    distances = distance.cdist(testingVector, baseData, "cosine")[0]
    min_index = np.argmin(distances)
    min_distance = distances[min_index]
    max_similarity = 1 - min_distance
    result.append(max_similarity)
    ndcga.append((ndcg_score(np.asarray(testingVector), np.asarray([baseData[min_index]]))))
    index = filteredDocstring.index(doc)
    closest = closest_doc_indexes[index]
    best = np.argsort(distances)[:50]
    
    if closest in best:
      best = list(best)
      mrr.append(1/(best.index(closest)+1))
    else:
      mrr.append(0)
  

(10000, 5)


10000it [00:14, 696.42it/s]
/var/folders/b0/nlrr4t_x3j5_jvwwktffc3j00000gn/T/ipykernel_52588/323944334.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trn_sub['code token'] = codeToken
/var/folders/b0/nlrr4t_x3j5_jvwwktffc3j00000gn/T/ipykernel_52588/323944334.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trn_sub['doc token'] = docToken
/var/folders/b0/nlrr4t_x3j5_jvwwktffc3j00000gn/T/ipykernel_52588/323944334.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [4]:
result = [x for x in result if str(x) != 'nan']

print('NDCGA Results : ',sum(ndcga)/len(ndcga))
print('Similarity Average : ',sum(result)/len(result))
print('Mean Reciprocal Rank :  ',sum(mrr)/len(mrr))

0.5273037586640298
0.38574048077399675
0.031637787403105605
